In [ ]:
# our database design must enforce the functional dependencies and referential integrity, 
#based on the following assumptions:

# Every company has an ID number, name, headquarter city, CEO, and the founding year
# Every CEO has an ID, first and last name, year of birth, place of birth, the year when they became a CEO
# Every city is located in a state, every state in a country. In some cases, there are no states.

In [ ]:
#install the library to connect pgAdmina
!pip install -U "psycopg[binary]"

In [1]:
import psycopg, os

print('Connecting to the PostgreSQL database...')

conn = psycopg.connect(
    host="localhost",
    port='5432',
    dbname="postgres",
    user="postgres",
    password="123")

Connecting to the PostgreSQL database...


In [2]:
cur = conn.cursor()

In [3]:
# execute a statement
print('PostgreSQL database version:')
cur.execute('SELECT version()')

# display the PostgreSQL database server version
db_version = cur.fetchone()
print(db_version)

PostgreSQL database version:
('PostgreSQL 15.1 (Debian 15.1-1.pgdg110+1) on aarch64-unknown-linux-gnu, compiled by gcc (Debian 10.2.1-6) 10.2.1 20210110, 64-bit',)


In [4]:
cur.execute("DROP TABLE company")

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost database=postgres) at 0x7ff2791abb40>

In [5]:
cur.execute("DROP TABLE ceo")

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost database=postgres) at 0x7ff2791abb40>

In [6]:
cur.execute("DROP TABLE cities")

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost database=postgres) at 0x7ff2791abb40>

In [7]:
createCmd = """ create table cities (
                city_name  varchar(255),
                state  varchar(255),
                country  varchar(255),
                primary key (city_name))
            """
    
cur.execute(createCmd)
conn.commit()

In [8]:
createCmd = """ create table ceo (
                id  varchar(255),
                name  varchar(255),
                year_of_birth numeric(4,0),
                city_name varchar(255),
                year_become_ceo numeric(4,0),
                primary key (name),
                foreign key (city_name) references cities(city_name))
            """
    
cur.execute(createCmd)
conn.commit()

In [9]:
createCmd = """ create table company (
                id  varchar(20),
                name  varchar(255), 
                city_name  varchar(255),
                ceo_name  varchar(255),
                year numeric(4,0),
                primary key (id),
                foreign key (ceo_name) references ceo(name),
                foreign key (city_name) references cities(city_name))
            """
    
cur.execute(createCmd)
conn.commit()

In [10]:
queryCmd = 'SELECT * from ceo'
cur.execute(queryCmd)
print(cur.rowcount)

0


In [11]:
# city table
# city, state, country

cur.execute("INSERT into cities VALUES('Mobile', 'AL', 'USA')")
cur.execute("INSERT into cities VALUES('Cupertino', 'CA', 'USA')")
cur.execute("INSERT into cities VALUES('West Godavari', 'Andhra-Pradesh', 'India')")
cur.execute("INSERT into cities VALUES('Armonk', 'NY', 'USA')")
cur.execute("INSERT into cities VALUES('Scarsdale', 'NY', 'USA')")
cur.execute("INSERT into cities VALUES('Seattle', 'WA', 'USA')")
cur.execute("INSERT into cities VALUES('Royal Oak', 'MI', 'USA')")
cur.execute("INSERT into cities VALUES('Detroit', 'MI', 'USA')")
cur.execute("INSERT into cities VALUES('Kumamoto', '', 'Japan')")
cur.execute("INSERT into cities VALUES('Tokyo', '', 'Japan')")
cur.execute("INSERT into cities VALUES('Shanghai', '', 'China')")
cur.execute("INSERT into cities VALUES('Hangzhou', 'Zhejiang', 'China')")
cur.execute("INSERT into cities VALUES('Rennes', 'Illes-et-Villaine', 'France')")
cur.execute("INSERT into cities VALUES('Paris', '', 'France')")
cur.execute("INSERT into cities VALUES('Suwon', 'Gyeonggi-do', 'South Korea')")
cur.execute("INSERT into cities VALUES('Seoul', '', 'South Korea')")
cur.execute("INSERT into cities VALUES('Memphis', 'TN', 'USA')")
cur.execute("INSERT into cities VALUES('Bentonville', 'Arkansas', 'USA')")

conn.commit()

In [12]:
# ceo table
#id, ceo, birth, city, country, year become ceo
cur.execute("INSERT into ceo VALUES(1, 'Tim Cook', 1960, 'Mobile', 2011)")
cur.execute("INSERT into ceo VALUES(2, 'Arvind Krishna', 1962, 'West Godavari', 2020)")
cur.execute("INSERT into ceo VALUES(3, 'Andy Jassy', 1968, 'Scarsdale', 2021)")
cur.execute("INSERT into ceo VALUES(4, 'Mary Barra', 1961, 'Royal Oak', 2014)")
cur.execute("INSERT into ceo VALUES(5, 'Kenichiro Yoshida', 1959, 'Kumamoto', 2018)")
cur.execute("INSERT into ceo VALUES(6, 'Daniel Zhang', 1972, 'Shanghai', 2015)")
cur.execute("INSERT into ceo VALUES(7, 'François-Henri Pinault', 1962, 'Rennes', 2005)")
cur.execute("INSERT into ceo VALUES(8, 'Lee Jae-yong', 1968, 'Seoul', 2022)")
cur.execute("INSERT into ceo VALUES(9, 'Doug McMillon', 1966, 'Memphis', 2014)")

conn.commit()

In [13]:
# company table
# id,company,headquater,ceo,founded
cur.execute("INSERT into company VALUES(1, 'Apple', 'Cupertino', 'Tim Cook', 1976)")
cur.execute("INSERT into company VALUES(2, 'IBM', 'Armonk', 'Arvind Krishna', 1911)")
cur.execute("INSERT into company VALUES(3, 'Amazon', 'Seattle', 'Andy Jassy', 1994)")
cur.execute("INSERT into company VALUES(4, 'General Motors', 'Detroit', 'Mary Barra', 1908)")
cur.execute("INSERT into company VALUES(5, 'Sony', 'Tokyo', 'Kenichiro Yoshida', 1946)")
cur.execute("INSERT into company VALUES(6, 'Alibaba', 'Hangzhou', 'Daniel Zhang', 1999)")
cur.execute("INSERT into company VALUES(7, 'Kering', 'Paris', 'François-Henri Pinault', 1963)")
cur.execute("INSERT into company VALUES(8, 'Samsung', 'Suwon', 'Lee Jae-yong', 1969)")
cur.execute("INSERT into company VALUES(9, 'Walmart', 'Bentonville', 'Doug McMillon', 1962)")

conn.commit()

In [16]:
conn.commit()

In [14]:
#1 What’s the age of the company whose CEO was born in the state of Tennessee?

cmd = "SELECT year_of_birth FROM ceo JOIN cities ON ceo.city_name = cities.city_name WHERE cities.state = 'TN'"
cur.execute(cmd)

row = cur.fetchone()
year_of_birth = int(row[0])
age = 2023 - year_of_birth 
print(age)

57


In [18]:
#2 What are the full names of CEOs of companies founded in the USA before 1965?

cmd = "SELECT ceo_name FROM company JOIN cities ON company.city_name = cities.city_name WHERE cities.country = 'USA' AND company.year < 1965"
cur.execute(cmd)

row = cur.fetchone()
while row is not None:
    print(row[0])
    row = cur.fetchone()

Arvind Krishna
Mary Barra
Doug McMillon


In [19]:
#3 Which companies have CEOs who are younger than 60?

cmd = "SELECT company.name FROM company JOIN ceo ON company.ceo_name = ceo.name WHERE ceo.year_of_birth  > 1963 "
cur.execute(cmd)

row = cur.fetchone()
while row is not None:
    print(row[0])
    row = cur.fetchone()

Amazon
Alibaba
Samsung
Walmart


In [20]:
#4 What is the headquarter city of the company whose CEO was born in India? 
cmd = "SELECT company.city_name FROM company JOIN ceo ON company.ceo_name = ceo.name JOIN cities ON ceo.city_name = cities.city_name WHERE cities.country = 'India'"
        
cur.execute(cmd)

row = cur.fetchone()
while row is not None:
    print(row[0])
    row = cur.fetchone()

Armonk


In [21]:
#5 Which companies have CEOs who started in their positions after 2014?

cmd = "SELECT company.name FROM company JOIN ceo ON company.ceo_name = ceo.name WHERE year_become_ceo  > 2014 "
cur.execute(cmd)

row = cur.fetchone()
while row is not None:
    print(row[0])
    row = cur.fetchone()

IBM
Amazon
Sony
Alibaba
Samsung


In [22]:
cur.close()